NOTE: Before running this notebook, one may need to install the packages below if using Jupyterlab
```bash
conda install -c conda-forge nodejs
jupyter labextension install @jupyter-widgets/jupyterlab-manager
```

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import spectrum
from pyleoclim import Spectral
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter
import LMRt
from tqdm import tqdm

from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [39]:
def run_wwz(series, xlim=[0, 0.05], factor=1e3, label='WWZ', c=1e-3, loglog=False, title=None):
    time = series.index.values
    signal = series.values
    tau = np.linspace(np.min(time), np.max(time), 21)
    res_psd = Spectral.wwz_psd(signal, time, tau=tau, c=c, nMC=0, standardize=False)
    
    med = np.nanmedian(res_psd.psd)
    print('median PSD:', med)
    threshold = factor*med
    print('thread PSD:', threshold)
    activated_freqs = res_psd.freqs[res_psd.psd > threshold]
    print('window:', np.max(activated_freqs) - np.min(activated_freqs))
    
    sns.set(style='ticks', font_scale=1.5)
    fig = plt.figure(figsize=[10, 10])                                                                                                                                           
    ax_signal = plt.subplot(2, 1, 1)
    ax_signal.plot(time, signal, label='signal')                                                                                                                                                          
    ax_signal.spines['right'].set_visible(False)
    ax_signal.spines['top'].set_visible(False)
    ax_signal.set_ylabel('Value')
    ax_signal.set_xlabel('Time')
    if title:
        ax_signal.set_title(title)
    
    ax_spec = plt.subplot(2, 1, 2)
    if loglog:
        ax_spec.loglog(res_psd.freqs, res_psd.psd, lw=3, label=label)                                                                                                                                                          
    else:
        ax_spec.plot(res_psd.freqs, res_psd.psd, lw=3, label=label)                                                                                                                                                          
        ax_spec.set_xlim(xlim)
        
    ax_spec.axhline(y=threshold, ls='--', lw=2, color='k')
    ax_spec.get_xaxis().set_major_formatter(ScalarFormatter())
    ax_spec.xaxis.set_major_formatter(FormatStrFormatter('%g'))
    ax_spec.set_ylabel('Spectral Density')                                                                                                                                                 
    ax_spec.set_xlabel('Frequency')                                                                                                                                                   
    ax_spec.legend(frameon=False)
    ax_spec.spines['right'].set_visible(False)
    ax_spec.spines['top'].set_visible(False)
    
#     return fig, res_psd

# creat a DataFrame to store the data
df = pd.DataFrame()

In [40]:
time1 = np.arange(1000)
f1 = 1/50
signal1 = np.cos(2*np.pi*f1*time1)

time2 = np.arange(1000, 2001)
f2 = 1/60
signal2 = np.cos(2*np.pi*f2*time2)

signal = np.concatenate([signal1, signal2])
time = np.concatenate([time1, time2])

# series = pd.Series(signal, index=time)
series = pd.Series(signal1, index=time1)
df['two_freqs'] = series

def interact_wwz(c, factor):
    print(f'c={c}, factor={factor}')
    return run_wwz(series, c=c, factor=factor)

In [41]:
c = widgets.FloatSlider(min=1e-4, max=0.01, step=1e-4)
factor = widgets.FloatSlider(min=1e3, max=1e5, step=1e3)
widgets.interact(interact_wwz, c=c, factor=factor)

interactive(children=(FloatSlider(value=0.0001, description='c', max=0.01, min=0.0001, step=0.0001), FloatSlid…

<function __main__.interact_wwz(c, factor)>